# Multi-task Learning Model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append("..")
import math
import os 
import numpy as np
from torch.utils.data import Dataset,DataLoader,SubsetRandomSampler
from torchvision import transforms
import torchvision.transforms.functional as F
from models.resnext import *
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import cv2
%matplotlib inline

In [3]:
net = ResNeXt29_2x64d()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)

In [9]:
class CreateDataset(Dataset):
    def __init__(self, df_data, data_dir = PATH, transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name+'.tif')
        im = cv2.imread(img_path)
        image = cv2.resize(im,(32,32))
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    

In [11]:
class Flatten(nn.Module):
    def forward(self, input):        
        return input.view(input.size(0), -1)
    
class MakeNetwork(nn.Module):
    def __init__(self,units):
        super(MakeNetworks,self).__init__()
        self.units = units
        self.m = list(net.children())[0]    
        self.mbase = nn.Sequential(*list(self.m.children())[:-1])              
        self.mds = nn.ModuleList([self.mh(unit) for unit in self.units])
    def model_heads(self,n = 2):
        head = nn.Sequential(nn.AdaptiveAvgPool2d(2),
                           Flatten(),
                           nn.Linear(1024*2*2,512),
                           nn.Dropout(0.5)
                           nn.BatchNorm1d(512),
                           nn.ReLU(inplace=True),
                           nn.Dropout(0.5),
                           nn.Linear(512,128),
                           nn.BatchNorm1d(128),
                           nn.ReLU(inplace=True),
                           nn.Linear(128,n))
        return nn.Sequential(*list(head.children()))
    
    def forward(self,x,idx): 
        base = self.mbase(x)      
        for ix,m in enumerate(self.mds.to(device)):
            if idx[0]==ix:
                model = m     
        return model(base)
    

In [ ]:
mtl_model = MakeNetworks([2,2,7]);mtl_model = mtl_model.to(device)